In [1]:
#Combined NoteBook

! pip install pandas
! pip install openpyxl
!pip install rdflib
import pandas as pd
import rdflib
import hashlib
import numpy as np
import urllib.parse
from rdflib import Literal, Namespace, RDF, URIRef, Graph, RDFS, OWL
from rdflib.namespace import FOAF, XSD
from rdflib.plugins.sparql import prepareQuery
from pyspark.sql.functions import when, col, lit
from datetime import datetime

In [2]:
# Data creation and loading
# Load data from CSV files
lab1 = pd.read_csv("output/PigLab1_query_results.csv")
lab2 = pd.read_csv("output/PigLab2_query_results.csv")
lab3 = pd.read_csv("output/PigLab3_query_results.csv")
lab4 = pd.read_csv("output/PigLab4_query_results.csv", low_memory=False)
lab5 = pd.read_csv("output/PigLab5_query_results.csv")
lab6 = pd.read_csv("output/PigLab6_query_results.csv")


In [3]:
# Combine all lab data into a single DataFrame
barometer_tableau = pd.concat([lab1, lab2, lab3, lab4, lab5, lab6], ignore_index=True)

barometer_tableau

,SampleNumber,Province,FlooredDate,Pathogen,Result,SampleType,DiagnosticTest,Country,Breed,LabReference,PigStage,AutonomousCommunity,Month,Year
0,Lab1PigSample_0,Missing,2021-01-05,Swine influenza virus,1,Tissue,PCR,Spain,Meat,1,Missing,NaN,1,2021
1,Lab1PigSample_1,Huesca,2021-01-05,Swine influenza virus,0,Tissue,PCR,Spain,Meat,1,Sow,NaN,1,2021
2,Lab1PigSample_2,Missing,2021-01-05,Swine influenza virus,0,Tissue,PCR,Spain,Meat,1,Missing,NaN,1,2021
3,Lab1PigSample_3,Missing,2021-01-05,Swine influenza virus,0,Tissue,PCR,Spain,Meat,1,Missing,NaN,1,2021
4,Lab1PigSample_4,Missing,2021-01-08,Swine influenza virus,0,Fluid,PCR,Spain,Meat,1,Missing,NaN,1,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376618,Lab1PigSample_4286,Burgos,2024-09-12,Porcine epidemic diarrhea virus,0,Swab,PCR,Spain,Meat,6,Finishing,NaN,9,2024
376619,Lab1PigSample_4287,Segovia,2024-10-01,Porcine epidemic diarrhea virus,0,Faeces,PCR,Spain,Meat,6,Finishing,NaN,10,2024
376620,Lab1PigSample_4288,Segovia,2024-10-02,Porcine epidemic diarrhea virus,0,Faeces,PCR,Spain,Meat,6,Finishing,NaN,10,2024
376621,Lab1PigSample_4289,Ourense,2024-10-18,Porcine epidemic diarrhea virus,0,Faeces,PCR,Spain,Meat,6,Missing,NaN,10,2024


In [4]:
print(barometer_tableau['LabReference'].unique())

[1 2 3 4 5 6]


In [5]:
print(barometer_tableau['Pathogen'].unique())

['Swine influenza virus' 'Lawsonia intracellularis'
 'Mycoplasma hyopneumoniae' 'Actinobacillus pleuropneumoniae'
 'Porcine reproductive and respiratory syndrome virus'
 'Porcine epidemic diarrhea virus' 'Brachyspira hyodysenteriae'
 'Brachyspira spp']


In [10]:
print(barometer_tableau['Year'].unique())

[2021 2022 2023 2024 2020 2025]


In [7]:
print(barometer_tableau['SampleType'].unique())

['Tissue' 'Fluid' 'Mixed' 'Faeces' 'Swab' 'Other' 'Blood' 'Missing' 'BAL']


In [8]:
print(barometer_tableau['PigStage'].unique())

['Missing' 'Sow' 'Nursery' 'Finishing' 'Gilt' 'Other' 'Weaning' 'Mixed'
 'Suckling' 'Boar']


In [9]:
print(barometer_tableau['Province'].unique())

['Missing' 'Huesca' 'Zaragoza' 'Lleida' 'Teruel' 'Barcelona' 'Navarre'
 'Tarragona' 'Malaga' 'Castellon' 'Almeria' 'Ourense' 'Soria' 'Girona'
 'Murcia' 'Albacete' 'LaRioja' 'Granada' 'Valencia' 'Sevilla' 'Badajoz'
 'Segovia' 'Cuenca' 'Toledo' 'Caceres' 'Zamora' 'Cordoba' 'Salamanca'
 'Valladolid' 'LasPalmas' 'Burgos' 'Avila' 'Cadiz' 'Alicante' 'Palencia'
 'Pontevedra' 'Lugo' 'Huelva' 'ACoruna' 'Gipuzkoa' 'Alava' 'CiudadReal'
 'Bizkaia' 'Portugal' 'IslasBaleares' 'Madrid' 'GranCanaria' 'Jaen' 'Leon'
 'SantaCruzDeTenerife' 'Asturias' 'Guadalajara' 'Melilla' 'Ceuta'
 'Cantabria']


In [35]:
# Save file with the current date
current_date = datetime.today().strftime('%Y-%m-%d')
base_filename = "barometer_combined"
folder_path = "output/CleanedData/"
new_filename = f"{folder_path}{base_filename}_{current_date}.csv"

barometer_tableau.to_csv(new_filename, index=False)